<a href="https://colab.research.google.com/github/jorgeecr/loan/blob/master/faxinas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import cvxpy as cp
import matplotlib
import scipy

In [0]:
from matplotlib import pyplot as plt
matplotlib.style.use('seaborn')

In [0]:
# Data utilities
def index_by_date(df):
    try:
        df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y')
    except:
        df['date'] = pd.to_datetime(df['date'], format='%Y/%m/%d')
    df = df.set_index('date')
    
    return df

def adjust_returns(spot_return, decay):
    spot_momentum = spot_return.ewm(com=decay / (1.0 - decay), adjust=False).mean()
    adj_return = spot_return - spot_momentum
    
    return adj_return

In [0]:
path = '/content/'
usd_ret = pd.read_csv('{}usd_returns.csv'.format(path))
chf_ret = pd.read_csv('{}chf_returns.csv'.format(path))
premia = pd.read_csv('{}premia_returns.csv'.format(path))
usd_ret = index_by_date(usd_ret)
chf_ret = index_by_date(chf_ret)
premia = index_by_date(premia)

In [0]:
# Here you could select only the ccy pairs you want in the portfolio, usd or chf base
start_sim = '2008-01-01'
portfolio_returns = usd_ret.copy()
portfolio_returns['target'] =  premia['target']
portfolio_returns = portfolio_returns[start_sim:]

In [6]:
portfolio_returns.head()

,AUDUSD,EURUSD,GBPUSD,NZDUSD,CADUSD,CHFUSD,JPYUSD,NOKUSD,SEKUSD,target
date,,,,,,,,,,
2008-01-01,0.001906,0.001691,0.002161,0.000042,-0.005947,0.003084,0.003162,0.000562,0.003022,-0.000776
2008-01-02,0.002467,0.008436,-0.003382,0.008366,0.006531,0.014324,0.018418,0.007301,0.007972,-0.000891
2008-01-03,0.001785,0.000666,-0.002535,0.003269,0.002218,0.002788,-0.002068,0.009821,0.007410,-0.003306
2008-01-04,-0.008456,0.001686,-0.001524,-0.007743,-0.012973,0.004964,0.009203,0.002504,0.001890,-0.000407
2008-01-07,-0.003232,-0.005438,-0.000355,-0.001786,-0.003291,-0.009693,-0.005382,-0.007994,-0.007615,0.002526


In [0]:
# Constants
decay = 0.97
w_max = 1
w_min = -1
spars_param = 1 / 1000000

In [0]:
port_adjRet = adjust_returns(portfolio_returns, decay)
cov_matr_arr = port_adjRet.ewm(com=decay / (1.0 - decay), adjust=False).cov(bias=True)
m, n = len(cov_matr_arr.index.levels[0]), len(cov_matr_arr.index.levels[1])
cov_matr_arr = cov_matr_arr.values.reshape(m, n, -1).swapaxes(1, 2)
time = 3000
covMat = cov_matr_arr[time]

Sean:

In [0]:
C = newcov = covMat[:9,:9]
q = covMat[9][:9]

Por ser una forma cuadrática, C = newcov alcanza un mínimo igual a su eigenvalor más chico (https://math.stackexchange.com/questions/728037/minimum-of-a-quadratic-form), es decir:

$eigen \leq x^T C x$ para toda $x$, donde

In [0]:
eigen = min(np.linalg.eig(newcov)[0])

Tomando inversos, esto implica que $\frac{1}{x^T C x} \leq \frac{1}{eigen}$

Consideremos $w_i[Cw]_i$. Como los pesos son $\leq 1$, tenemos que 

$w_i[Cw]_i \leq 1*[Cw]_i \leq [Cw]_i$

El término de la derecha de esta desigualdad puede escribirse como $[Cw]_i = C[i]^T \cdot x$, donde $C[i]$ es la $i$-ésima fila de C.  

Hagamos $b = C[i]$, entonces reescribiendo lo anterior tenemos que $w_i[Cw]_i \leq b^T x$

Si tuviéramos que $b^T x \leq k$, para algún $k$ entonces obtendriamos como consecuencia: $\frac{w_i[Cw]_i}{x^T C x} \leq  \frac{b^T x}{x^T C x} \leq \frac{k}{eigen}$ 

Haciendo $k = 0.4 * eigen$ obtenemos $\frac{w_i[Cw]_i}{x^T C x} \leq 0.4$, que es precisamente lo que queremos. En resumen, la condición que necesitamos es  $b^T x \leq 0.4 * eigen$